In [ ]:
!pip install pyarabic


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.0 MB/s eta 0:00:00


In [ ]:

import pandas as pd
import pyarabic.araby as araby
file_path = 'synset_sinalab.csv'

df = pd.read_csv(file_path)
df

,synonyms,average,word
0,قَضِيَّة,40,شَأن
1,مَسْألَة,75,شَأن
2,أمْر,"72,5",شَأن
3,شَيء,80,شَأن
4,شَهْر شَمْسِيّ,"72,5",شَهْر
...,...,...,...
2999,نَوَوِيّ,"47,5",ذَرِّيّ
3000,نَوَوِي,"47,5",ذَرِّيّ
3001,ذَرِّي,100,ذَرِّيّ
3002,خُصُوصَاً,95,بِشَكْل خَاص;بِطَرِيقَة خَاصَّة


In [ ]:
import requests

api= "fee6626b-fc57-443d-a838-60d0de172316"


headers = {
    'accept': 'application/json',
    'apikey': api
}
def fun_request(response):
    try:
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            data = response.json()
            # Process the data as needed

            return data
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None  # Return None when there is an error
    except requests.exceptions.RequestException as e:
        print(f"Request Exception: {e}")
        return None  # Return None when there is a request exception)


In [ ]:
def get_synonems(ID):
  url = 'https://siwar.ksaa.gov.sa/api/alriyadh/find/entries/by/senses/ids'
  headers = {
    'accept': 'application/json',
    'apikey': 'fee6626b-fc57-443d-a838-60d0de172316',
    'senseId': ID,
    'Content-Type': 'application/json'
  }
  data = {"ids": [ID]}

  response = requests.patch(url, headers=headers, json=data)
  lemma_forms = [item['lemma']['formRepresentations'][0]['form'] for item in response.json()]
  return lemma_forms

In [ ]:
def get_domains(response):
      url =  "https://siwar.ksaa.gov.sa/api/alriyadh/find/lookups"
      response = requests.get(url, headers=headers)
      data= fun_request(response)
def search_word(word):
     url = 'https://siwar.ksaa.gov.sa/api/alriyadh/search'
     params = { 'query':word }
     response = requests.get(url, headers=headers, params=params)
     data= fun_request(response)

     for entry in data:
       for sense in entry.get('senses', []):
        definition = sense.get('definition', {}).get('textRepresentations', [])[0].get('form', '')
        print(definition)
        extracted_data=[]
        for relation in sense.get('relations', []):
                relation_type = relation.get('type', '')
                related_id = relation.get('related', '')
                relation=get_synonems(related_id)
                extracted_data.append({'RelationType': relation_type, 'word': relation})
        print(extracted_data)
     extracted_data = []
     for stem in entry.get('stems', []):
            stem_type = stem.get('type', '')
            stem_form = stem.get('formRepresentations', [{}])[0].get('form', '')
            extracted_data.append({'StemType': stem_type, 'StemForm': stem_form})
            print(extracted_data)

word= "كبير"
#COIN : تراكيب
#SYNDROM :
#ANTON : ضد
#SYNON: مرادف
search_word(word)


ذو حجم ضخم‏.
[]
مَن زادت سنه؛ بَلَغَ مبلغَ الرِّجال.
[]
عظيم، مهم.
[{'RelationType': 'COIN', 'word': ['كَبِير اَلْيَاوَرَانِ']}]
رئيس أو زعيم.
[{'RelationType': 'SYNON', 'word': ['قِيَادِيّ']}, {'RelationType': 'SYNON', 'word': ['قَائِد']}, {'RelationType': 'SYNDROM', 'word': ['كَبِير اَلْعَائِلَةِ']}]
ذنب عظيم عليه عقوبة شرعية مثل القتل وشرب الخمر.
[]
الآمر الناهي في العائلة.
[]
شخص مسؤول عن إدارة بيت رئيس الدولة.
[]
مؤنث (كَبِير)
[]
[{'StemType': 'root', 'StemForm': 'ك ب ر'}]


In [ ]:
import re


def extract_arabic_word(text):
    if isinstance(text, str):  # Check if the value is a string
        match = re.match(r'([^\|]+)\|', text)
        if match:
            return match.group(1).strip()
    return None

In [ ]:
import csv
csv_file_path = 'domains_data.csv'

# Open the CSV file in write mode
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    # Create a CSV writer
    csv_writer = csv.writer(csv_file)

    # Write the header
    header = ['_id', 'code', 'label', 'category', 'order', 'updatedAt', 'updatedByUserId']
    csv_writer.writerow(header)

    # Write the data
    for domain in data['domains']:
        csv_writer.writerow([domain['_id'], domain['code'], domain['label'], domain['category'],
                             domain['order'], domain['updatedAt'], domain['updatedByUserId']])

print(f"Data has been written to {csv_file_path}")

In [ ]:
import csv

# Replace 'your_input.txt' with the actual path to your .txt file
input_file_path = 'ARCOQ.txt'

# Replace 'your_output.csv' with the desired path for the CSV output file
output_file_path = 'ANTONYM.csv'

# Read data from the .txt file
with open(input_file_path, 'r', encoding='utf-8') as txt_file:
    # Assuming each line in the file contains the lemma, words, and correct answer separated by '::'
    lines = [line.strip().split(' :: ') for line in txt_file if line.strip()]

# Write data to a CSV file
with open(output_file_path, 'w', encoding='utf-8', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(lines)

print(f"Conversion from {input_file_path} to {output_file_path} completed.")


In [ ]:


before_filter="اللَّهمَّ اغْفِرْ لنَا ولوالدِينَا"
after_filter = araby.strip_diacritics(before_filter)

print(after_filter)
# will print : اللهم اغفر لنا ولوالدينا